# Mesos Lab

First move to the source directory where you already downloaded the git repo sources:

git clone https://github.com/mjbright/LinuxConEU-ContainerOrchestration


In [1]:
pwd

. ../NB_bash_functions.rc

CLUSTER_OPTION=1
#CLUSTER_OPTION=2

SCRIPT_DIR=~/PYCONUS_DOWNLOADS/scripts
export PATH=$SCRIPT_DIR:$PATH

/home/mjb/Dropbox/z/bin/DEMOS_TUTORIALS/2017-May-PyconUS/mesos-cli
Notebook started at Mon May 15 11:08:49 CEST 2017   [1494839329]


## Cleanup the Mesos Cluster

In [2]:
time docker-machine rm -f mesosmaster mesosslave1 mesosslave2

NB_cleanup_virtualbox_inaccessible_VMs
NB_cleanup_virtualbox_mesos_VMs

About to remove mesosmaster, mesosslave1, mesosslave2
Successfully removed mesosmaster
Error removing host "mesosslave1": Host does not exist: "mesosslave1"
Can't remove "mesosslave1"
Error removing host "mesosslave2": Host does not exist: "mesosslave2"
Can't remove "mesosslave2"

real	0m2.656s
user	0m0.028s
sys	0m0.004s


## Create the Mesos Cluster

We will now use docker-machine/virtualbox to create the Docker hosts to house Mesos.

We can either
- create 1 machine which runs Mesos Master and several Slaves, or
- create 1 Master machine and several slave machines

For this lab we recommend the first option - 1 single machine

In either case we will first create a Docker host machine called mesosmaster

In [3]:
[ $CLUSTER_OPTION -eq 1 ] && time docker-machine create --driver virtualbox mesosmaster

Running pre-create checks...
Creating machine...
(mesosmaster) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/mesosmaster/boot2docker.iso...
(mesosmaster) Creating VirtualBox VM...
(mesosmaster) Creating SSH key...
(mesosmaster) Starting the VM...
(mesosmaster) Check network to re-create if needed...
(mesosmaster) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env mesosmaster

real	1m48.302s
user	0m14.604s
sys	0m32.536s


In [4]:
docker-machine ls

NAME          ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
mesosmaster   -        virtualbox   Running   tcp://192.168.99.123:2376           v17.05.0-ce   
swmaster1     -        virtualbox   Running   tcp://192.168.99.114:2376           v17.05.0-ce   
swnode1       -        virtualbox   Running   tcp://192.168.99.115:2376           v17.05.0-ce   
swnode2       -        virtualbox   Running   tcp://192.168.99.116:2376           v17.05.0-ce   


### Start the Mesos Master - Option1: Single docker machine

#### Upload images

To save network bandwidth we will pre-load container images from a local directory

In [5]:
[ $CLUSTER_OPTION -eq 1 ] && time docker_load_save_images.sh -m mesosmaster load


Loading images to machine <tcp://192.168.99.123:2376 / mesosmaster> ...
-rw------- 1 mjb mjb 1.1G May 15 10:49 /home/mjb/PYCONUS_DOWNLOADS/Docker-images/mesosmaster/mesosphere____marathon____v0.8.1
IMAGE_FILE=mesosphere____marathon____v0.8.1
REPO=mesosphere
REST=marathon____v0.8.1
IMAGE=marathon
TAG=v0.8.1
docker load -i /home/mjb/PYCONUS_DOWNLOADS/Docker-images/mesosmaster/mesosphere____marathon____v0.8.1

aca5ce76: Loading layer  197.2MB/197.2MB
d576a927: Loading layer  208.9kB/208.9kB
2f49bc4c: Loading layer  4.608kB/4.608kB
bf18a086: Loading layer  1.024kB/1.024kB
b6655687: Loading layer  21.26MB/21.26MB
46db564b: Loading layer  485.4MB/485.4MB
b10cef78: Loading layer  4.391MB/4.391MB
bf18a086: Loading layer  1.024kB/1.024kB
Loaded image: mesosphere/marathon:v0.8.1
Took 177 secs
-rw------- 1 mjb mjb 719M May 15 10:39 /home/mjb/PYCONUS_DOWNLOADS/Docker-images/mesosmaster/mesosphere____mesos-master____1.1.01.1.0-2.0.107.ubuntu1404
IMAGE_FILE=mesosphere____mesos-master____1.1.01.1.0-

We will now start the cluster (Mesos Master and 2 Slaves)

In [6]:
[ $CLUSTER_OPTION -eq 1 ] && time docker-compose $(docker-machine config mesosmaster) -f docker-compose.yml up -d

Creating mesoscli_slave2_1 ... 
Creating mesoscli_mesos1_1 ... 
Creating mesoscli_zookeeper_1 ... 
Creating mesoscli_marathon_1 ... 
Creating mesoscli_slave1_1 ... 
Creating mesoscli_slave2_1
Creating mesoscli_zookeeper_1
Creating mesoscli_slave3_1 ... 
Creating mesoscli_mesos1_1
Creating mesoscli_slave1_1
Creating mesoscli_marathon_1
Creating mesoscli_slave3_1
 mesoscli_slave1_1 ... done0mm
real	0m23.999s
user	0m4.392s
sys	0m0.580s


### Start the Mesos Master - Option2: Separate docker machines house Master 2 Slaves

#### Option2: Create the Slave machines

In this scenario we will create separate machines for the Mesos Slaves.

We will now use docker-compose to start the Mesos Master and both Mesos Slaves as containers within the mesosmaster machine which we will created.

In [7]:
[ $CLUSTER_OPTION -eq 2 ] && time docker-machine create --driver virtualbox mesosslave1
NB_continue

In [8]:
[ $CLUSTER_OPTION -eq 2 ] && time docker-machine create --driver virtualbox mesosslave2
NB_continue

#### Upload images

To save network bandwidth we will pre-load container images from a local directory

#### Note: need to no how to only load mesosmaster images to master node etc ...

In [9]:
[ $CLUSTER_OPTION -eq 2 ] &&  {
    echo "Uploading images to mesosmaster"
    time docker_load_save_images.sh -m mesosmaster load

    echo "Uploading images to mesosslave1"
    time docker_load_save_images.sh -m mesosslave1 load

    echo "Uploading images to mesosslave2"
    time docker_load_save_images.sh -m mesosslave2 load
}
NB_continue

#### Option2: Start Mesos Master

We will now use docker-compose to start the Mesos Master as a container within the mesosmaster machine which we created

In [10]:
[ $CLUSTER_OPTION -eq 2 ] && time docker-compose $(docker-machine config mesosmaster) -f master/docker-compose.yml up -d
NB_continue

#### Option2: Start Mesos Slaves

We will now use docker-compose to start the Mesos Slaves as container within the mesosslave machines which we just created

In [11]:
[ $CLUSTER_OPTION -eq 2 ] && time docker-compose $(docker-machine config mesosslave1) -f slave/docker-compose.yml up -d
NB_continue

In [12]:
[ $CLUSTER_OPTION -eq 2 ] && time docker-compose $(docker-machine config mesosslave2) -f slave/docker-compose.yml up -d
NB_continue

### Setup the Mesos Slaves

We will now use docker-machine/virtualbox to create new docker nodes to be used as the Mesos Slaves.


## Access to the Marathon console from your laptop:

Let's get the ip address of our 'mesosmaster' machine, using
```bash
docker-machine ip mesosmaster
```

In [13]:
docker-machine ip mesosmaster

echo "From a terminal on your laptop
    docker-machine ssh mesosmaster -Nv -L 9090:localhost:8080""
    
Then connect to the Mesos dashboard at http://localhost:9090"

NB_pause

192.168.99.123
From a terminal on your laptop
    docker-machine ssh mesosmaster -Nv -L 9090:localhost:8080
    
Then connect to the Mesos dashboard at http://localhost:9090


: 2


Then open your web browser at the page http://localhost:9090 and you should see the Marathon console, as shown here

![](images/marathon_dashboard.png)

In [14]:
docker-machine ls

NAME          ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
mesosmaster   -        virtualbox   Running   tcp://192.168.99.123:2376           v17.05.0-ce   
swmaster1     -        virtualbox   Running   tcp://192.168.99.114:2376           v17.05.0-ce   
swnode1       -        virtualbox   Running   tcp://192.168.99.115:2376           v17.05.0-ce   
swnode2       -        virtualbox   Running   tcp://192.168.99.116:2376           v17.05.0-ce   


### Deploy applications on to the Mesos cluster

Use the Marathon application descriptions nginx.json and pacman.json to deploy these apps on mesos/marathon

Let's first obtain the ip address of our Mesos Master node:

In [15]:
MESOS_MASTERIP=$(docker-machine ip mesosmaster)
echo $MESOS_MASTERIP

192.168.99.123


Now let's launch the nginx application, by sending a POST request to the Mesos Master:

In [16]:
curl -X POST -H "Accept: application/json" -H "Content-Type: application/json" http://$MESOS_MASTERIP:8080/v2/apps -d @nginx.json

{"id":"/nginx","cmd":null,"args":null,"user":null,"env":{},"instances":2,"cpus":0.1,"mem":64.0,"disk":0.0,"executor":"","constraints":[],"uris":[],"storeUrls":[],"ports":[0],"requirePorts":false,"backoffFactor":1.15,"container":{"type":"DOCKER","volumes":[],"docker":{"image":"nginx","network":"BRIDGE","portMappings":[{"containerPort":80,"hostPort":0,"servicePort":0,"protocol":"tcp"}],"privileged":false,"parameters":[]}},"healthChecks":[{"path":"/","protocol":"HTTP","portIndex":0,"command":null,"gracePeriodSeconds":5,"intervalSeconds":20,"timeoutSeconds":20,"maxConsecutiveFailures":3}],"dependencies":[],"upgradeStrategy":{"minimumHealthCapacity":1.0,"maximumOverCapacity":1.0},"labels":{},"version":"2017-05-15T09:20:28.850Z","tasks":[],"deployments":[{"id":"b3971878-c9b3-4bb2-8e0f-7e61c820cfe6"}],"tasksStaged":0,"tasksRunning":0,"tasksHealthy":0,"tasksUnhealthy":0,"backoffSeconds":1,"maxLaunchDelaySeconds":3600}

and let's launch the pacman application, by sending a POST request to the Mesos Master:

In [17]:
curl -X POST -H "Accept: application/json" -H "Content-Type: application/json" http://$MESOS_MASTERIP:8080/v2/apps -d @pacman.json

{"id":"/pacman","cmd":null,"args":null,"user":null,"env":{},"instances":2,"cpus":0.1,"mem":64.0,"disk":0.0,"executor":"","constraints":[],"uris":[],"storeUrls":[],"ports":[0],"requirePorts":false,"backoffFactor":1.15,"container":{"type":"DOCKER","volumes":[],"docker":{"image":"emilevauge/pacman","network":"BRIDGE","portMappings":[{"containerPort":80,"hostPort":0,"servicePort":0,"protocol":"tcp"}],"privileged":false,"parameters":[]}},"healthChecks":[],"dependencies":[],"upgradeStrategy":{"minimumHealthCapacity":1.0,"maximumOverCapacity":1.0},"labels":{},"version":"2017-05-15T09:20:32.882Z","tasks":[],"deployments":[{"id":"18c20792-f692-4f36-a6fa-6ebf14aa6835"}],"tasksStaged":0,"tasksRunning":0,"tasksHealthy":0,"tasksUnhealthy":0,"backoffSeconds":1,"maxLaunchDelaySeconds":3600}

The Marathon console, at http://localhost:9090, should now show these 2 services to be running:

![](images/marathon_dashboard_2services.png)

Play with the Marathon console: scale applications, kill them etc...


In [18]:
NB_time_taken

Started at Mon May 15 11:20:33 CEST 2017   [1494839329]
Ended   at Mon May 15 11:20:33 CEST 2017   [1494840033]
Took 704 secs [0h11m44]
